In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from uncertainty_forest.uncertainty_forest import UncertaintyForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [ ]:
def generate_data(n, mean, var):
    y = 2 * np.random.binomial(1, .5, n) - 1 # classes are -1 and 1.
    X = np.random.multivariate_normal(mean * y, var * np.eye(n), 1).T
    return X, y

In [ ]:
# Estimate posteriors for many trials and evaluate in the given X_eval range.
def estimate_posterior(algo, n, mean, var, num_trials, X_eval, parallel = False):
    obj = algo['instance']
    def worker(t):
        X, y = generate_data(n, mean, var)
        obj.fit(X, y)
        class_1_index = np.argwhere(obj.classes_.ravel() == 1)[0][0]
        return obj.predict_proba(X_eval)[:, class_1_index]
        
    if parallel:
        predicted_posterior = np.array(Parallel(n_jobs=-2)(delayed(worker)(t) for t in range(num_trials)))
    else:
        predicted_posterior = np.zeros((num_trials, X_eval.shape[0]))
        for t in tqdm(range(num_trials)):
            predicted_posterior[t, :] = worker(t)

    return predicted_posterior

In [ ]:
# For either a CART, Honest, or Uncertainty Forest, plot P(Y = 1 | X = x).
def plot_posterior(ax, algo, num_plotted_trials, X_eval):
    # Plot each of num_plotted_trials iterations, highlighting a single line.
    for i in range(num_plotted_trials):
        linewidth = 1
        opacity = .3
        if i == num_plotted_trials - 1:
            opacity = 1
            linewidth = 8
        ax.set_title(algo['title'])
        ax.plot(X_eval.flatten().ravel(), algo['predicted_posterior'][i, :].ravel(), 
                label = algo['label'],
                linewidth = linewidth, 
                color = algo['color'], 
                alpha = opacity)

# Plot the variance over the number of trials.
def plot_variance(ax, algos, X_eval):
    ax.set_title('Posterior Variance')
    for algo in algos:
        variance = np.var(algo['predicted_posterior'], axis = 0)
        ax.plot(X_eval.flatten().ravel(), variance.ravel(), 
                label = algo['label'],
                linewidth = 8, 
                color = algo['color'])

In [ ]:
# Communal plotting parameters.
def plot_fig1(algos, num_plotted_trials):
    sns.set(font_scale = 6)
    sns.set_style("ticks")
    plt.rcParams['figure.figsize'] = [55, 14]
    fig, axes = plt.subplots(1, 4)
    for ax in axes[0:3]:
        ax.set_xlim(-2.1, 2.1)
        ax.set_ylim(-0.05, 1.05)

    # 3 posterior plots.
    for i in range(len(algos)):
        plot_posterior(axes[i], 
                       algos[i],
                       num_plotted_trials, 
                       X_eval)

    # 1 variance plot.
    # axes[3].set_ylim(-0.05, 0.1)
    plot_variance(axes[3], algos, X_eval)
    
    fig.text(0.5, .08, 'x', ha='center')
    axes[0].set_ylabel(r"$\hat P(Y = 1|X = x)$")
    axes[0].set_xlabel(" ")
    axes[3].set_ylabel(r"Var($\hat P(Y = 1|X = x)$)")
    
    fig.tight_layout()
    plt.savefig("fig1.pdf")
    plt.show()

In [ ]:
# Data.
n = 5000
mean = 1
var = 1
num_trials = 100
X_eval = np.linspace(-2, 2, num = 20).reshape(-1, 1)
n_estimators = 10

# Algorithms.
algos = [
    {
        'instance': RandomForestClassifier(n_estimators = n_estimators),
        'label': 'CART',
        'title': 'CART Forest',
        'color': "#1b9e77",
    },
    {
        'instance': CalibratedClassifierCV(base_estimator=RandomForestClassifier(n_estimators = n_estimators), 
                                           method='isotonic', 
                                           cv = 5),
        'label': 'IRF',
        'title': 'Isotonic Reg. Forest',
        'color': "#fdae61",
    },
    {
        'instance': UncertaintyForest(n_estimators = n_estimators),
        'label': 'UF',
        'title': 'Uncertainty Forest',
        'color': "#F41711",
    },
]

# Plotting parameters.
parallel = True
num_plotted_trials = 10

In [ ]:
# Generate data and predictions.
for algo in algos:
    algo['predicted_posterior'] = estimate_posterior(algo, n, mean, var, num_trials, X_eval, parallel = parallel)
    
with open('algos_fig1.pkl', 'wb') as f:
    pickle.dump(algos, f)

In [ ]:
with open('algos_fig1.pkl', 'rb') as f:
    algos = pickle.load(f)
    
plot_fig1(algos, num_plotted_trials)